In [1]:
## We will be using the Financial Modeling Prep API to fetch stock price data for this project. 

## For security reasons, the API key has been removed from this submission

In [33]:
%%sh 

pip install fmpsdk

In [34]:
import fmpsdk 
import pandas as pd 
import numpy as np

In [36]:
stocks = pd.read_csv("../data/SP500.csv")

In [28]:
stock_prices = []

stock_list = list(stocks.Symbol)

for i in range(0, len(stock_list), 5):
    shortlist = stock_list[i:i+5]
    stock_prices.extend(
        fmpsdk.historical_price_full(
            apikey = '### REDACTED ###', 
            symbol = shortlist, 
            from_date = '2023-10-01',
            to_date = '2024-04-01'
        ))



In [29]:
# This filters any stocks that were recent additions, or removals from the SP500;
# any stocks that have fewer than the expected number of trading days

stock_prices_f = [stock_prices[i] for i in range(len(stock_prices)) if len(stock_prices[i]['historical']) == 125]

In [30]:
len(stock_prices[0]['historical'])

125

In [31]:
names = []
daily_returns = []

for i in stock_prices_f:
    name = i['symbol']
    returns = pd.DataFrame(i['historical'])[['date', 'close']]
    returns = returns.rename({'close': name}, axis = 1)
    returns = returns.set_index('date')

    names.append(name)
    daily_returns.append(returns)

results = pd.concat(daily_returns, axis = 1)

corr_matrix = results.corr()

corr_matrix.to_csv("../data/correlation_matrix.csv")

In [32]:
corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape) - np.diag(np.diag(np.ones(corr_matrix.shape)))).astype(bool))

stacked = corr_matrix.stack().reset_index()

stacked = stacked.rename({'level_0': 'stock_a', 
                          'level_1': 'stock_b', 
                          '0': 'corr'}, axis = 1)

stacked.to_csv('../data/stacked_correlations_no_diag.csv')